# 新闻视频处理演示

在这个笔记本中，我们将要针对一则来自江西湖口的最近一段时间本地新闻进行如下的处理：

1. 按照 `cut-scene` 来切割出若干镜头

1. 对上一步切割出来的镜头进行分类，识别出:

  - 演播室（没有字幕）
  - 演播室（带有字幕）
  - 其它

1. 识别和切割出完整的独立新闻视频

1. 识别带有字幕的**非演播室**片段中的字幕文本

1. 识别**演播室**片段中，播音员播报的语音内容，将它转为文本

---

> 之所以选择来自湖口的最近新闻录像，是因为：
>
> 在这个例子中，我们所使用到的若干神经网络模型，均只使用了少量湖口的新闻视频的人工标注样本进行训练，数据量较小，难以泛化。

## 准备工作

> **❗ 重要：**
>
> 检查该 Notebook Web 界面的右上角，确定选择名为 `jxcn-newsvideo-poc` 的 `Kernel`
>
> 如果不是，可点击后在下拉列表中选中。

### 载入环境变量

这个笔记本的代码需要某些预定义的环境变量，所以 **必须** 加载。

**执行代码：**

In [ ]:
%load_ext dotenv

%dotenv -o ../.env

### 验证 模型运行环境

我们还需要：

1. 验证 PyTorch 的情况：使用 CPU 还是 GPU

1. 设置 matplotlib 中文字体：是否可以在绘图中显示中文

   请根据实际情况设置有效的字体！

1. 其他环境

**执行代码：**

In [ ]:
%matplotlib inline

import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as mfm
import torch

from wasabi import Printer

candidate_fonts = [
    "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc",  # ubuntu
    "/usr/share/fonts/google-noto-cjk/NotoSansCJK-Regular.ttc" # CentOS
]

msg = Printer()

with msg.loading("CJK plotting test..."):
    mfm_font_path = ''
    for font_path in candidate_fonts:
        if os.path.exists(font_path):
            mfm_font_path = font_path
            break
    if not mfm_font_path:
        raise RuntimeError('没有找到列表中的 CJK 字体')
    mfm_font_prop = mfm.FontProperties(fname=mfm_font_path)
    plt.text(.5, .5, s='中文绘图显示测试', fontproperties=mfm_font_prop)
    plt.pause(0.01)
    plt.show()
    plt.pause(0.01)

with msg.loading("Torch device checking ..."):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    # Assuming that we are on a CUDA machine, this should print a CUDA device
    display(pd.DataFrame([{
        'Pytorch Version': torch.__version__,
        'Pytorch Device': device
    }]))


### 定义全局函数

定义几个用于 A/V 和图像处理的全局函数

**执行代码：**

In [ ]:
import os
import math
import base64
import hashlib
import json
import urllib
import subprocess
from contextlib import contextmanager
from copy import copy, deepcopy
from datetime import timedelta
from functools import partial
from itertools import chain
from time import time, sleep

import numpy as np
import pandas as pd
import skimage
import matplotlib.pyplot as plt
import matplotlib.font_manager as mfm
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
from matplotlib.offsetbox import AnchoredText
from matplotlib.patheffects import withStroke
from mpl_toolkits.axes_grid1 import ImageGrid

import torch
import requests

from tqdm import tqdm_notebook as tqdm
from wasabi import Printer

from scenedetect.video_manager import VideoManager
from scenedetect.detectors import ContentDetector
from scenedetect.scene_manager import SceneManager
from scenedetect.stats_manager import StatsManager

from IPython.display import display, Image, Audio, Video, HTML


msg = Printer()


def run_ffmpeg(args=None, quiet=True, check=True, cmd=None):
    if not cmd:
        cmd = 'ffmpeg'
    args = args or []
    if quiet:
        kv_args = dict(stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    else:
        kv_args = dict()
    subprocess.run([cmd, '-y'] + args, check=check, **kv_args)

def trunc_av(video_file_path, start_seconds, end_seconds, output_path):
    start_time = timedelta(seconds=start_seconds)
    duration = timedelta(seconds=(end_seconds - start_seconds))
    args = [
        '-ss', str(start_time),
        '-i', str(video_file_path),
        '-t', str(duration),
        '-c', 'copy',
        # output:
        str(output_path)
    ]
    run_ffmpeg(args=args, quiet=False)

def trunc_a(video_file_path, start_seconds, end_seconds, output_path):
    start_time = timedelta(seconds=start_seconds)
    duration = timedelta(seconds=(end_seconds - start_seconds))
    args = [
        '-ss', str(start_time),
        '-i', str(video_file_path),
        '-t', str(duration),
        '-vn', '-acodec', 'libopus', '-b:a', '16k',
        # output:
        str(output_path)
    ]
    run_ffmpeg(args=args)


def add_inner_title(ax, title, loc, size=None, **kwargs):
    prop = kwargs.get('prop', {})
    prop['fontproperties'] = mfm_font_prop
    if size is None:
        prop['size'] = plt.rcParams['legend.fontsize']
#     at = AnchoredText(title, loc=loc, pad=0., borderpad=0.5, frameon=False, **kwargs)
    at = AnchoredText(title, loc=loc, prop=prop, pad=0., borderpad=0.5, frameon=False, **kwargs)
    ax.add_artist(at)
    at.txt._text.set_path_effects([withStroke(foreground="w", linewidth=3)])
    return at


## 上传要处理的视频文件

> **十分重要 :**
>
> 请操作者自行上传湖口最近一段时间的新闻录像，并修改下面的变量为真实的文件路径

上传一个湖口的新闻视频，并指定其路径。

本例中，我们上传文件 *湖口* *2019年05月16日* 的新闻录像到:

```sh
../videos/湖口/20190516.mp4
```

> **提示：**
>
> 笔记本 Web 界面左侧的文件浏览器支持拖拽

将文件路径赋值到全局变量 `VIDEO_DIR` 。后面的步骤需要使用该变量指定视频文件进行处理。

**执行代码：**

In [ ]:
import os

VIDEO_DIR = '../videos'  # 把视频上传到这个目录（相对于笔记本所在路径）

# 定义要处理的视频文件路径之全局变量
VIDEO_FILE = os.path.join(
    VIDEO_DIR,
    "湖口/20190516.mp4"  # 放到这个子目录！
)

print('要处理的视频：', os.path.abspath(VIDEO_FILE))  # 输出文件绝对路径
assert os.path.exists(VIDEO_FILE)  # 断言：文件存在
display(Video(VIDEO_FILE))

## 场景边界预测

这一小节中，我们需要根据视觉表征进行场景边界预测

在此之前，我们已经预先准备了一个使用根据视频内容表征进行视频场景边界检测的预训练模型。
这个模型可以预测出视频场景边界。

我们使用这个模型，对上传的新闻视频文件进行预测，得到它的场景边界，并显示缩略图

**执行代码：**

In [ ]:
msg.info('场景检测:')


@contextmanager
def managed_video_manager(*args, **kwargs):
    video_manager = VideoManager(*args, **kwargs)
    try:
        yield video_manager
    finally:
        video_manager.release()

def cut_scenes(video_file_path):
    # cut-scenes 检测
    stats_manager = StatsManager()
    scene_manager = SceneManager(stats_manager)
    scene_manager.add_detector(ContentDetector())
    
    with managed_video_manager([str(video_file_path)]) as video_manager:
        # detect scenes
        video_manager.start()
        base_time_code = video_manager.get_base_timecode()
        num_frames = scene_manager.detect_scenes(frame_source=video_manager)
        if not num_frames > 0:
            raise RuntimeError('No scenes detected')
        scene_list = scene_manager.get_scene_list(base_time_code)
    # Ok.
    # 拼凑结果数据
    scenes = []
    for index, (start_time_code, end_time_code) in enumerate(scene_list):
        scenes.append({
            'index': index,
            'start_time': start_time_code.get_seconds(),
            'end_time': end_time_code.get_seconds(),
            'start_frame': start_time_code.get_frames(),
            'end_frame': end_time_code.get_frames(),
        })
    return scenes

##

scenes = cut_scenes(VIDEO_FILE)

##########################################
# 生成缩略图：

from shutil import rmtree
from datetime import timedelta
from string import Template
    
def take_images_in_timespan(
        video_file_path,
        start_time,
        end_time,
        output_file_name_format='$index.jpg',
        num=3
):
    output_tpl = Template(output_file_name_format)
    time_points = []
    img_file_list = []
    if num == 1:
        time_points.append(
            start_time + (end_time - start_time) / 2
        )
    else:
        span = (end_time - start_time) / (num - 1)
        time_points.append(start_time)
        for i in range(num - 2):
            time_points.append(
                start_time + span * (i + 1)
            )
        time_points.append(end_time)
    #
    for i, ss in enumerate(time_points):
        ss = timedelta(seconds=ss)
        output_path = output_tpl.substitute(index=i + 1)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        run_ffmpeg([
            '-ss', str(ss), '-i', video_file_path, '-vframes', '1', output_path
        ])
        img_file_list.append(output_path)
    return img_file_list


def take_scenes_images(video_file_path, scenes, output_dir):
    # set output dir
    os.makedirs(output_dir, exist_ok=True)
    # take a frame as image
    for i, scene in tqdm(enumerate(scenes), total=len(scenes)):
        start_time = scene['start_time']
        end_time = scene['end_time']
        snapshots = take_images_in_timespan(
            video_file_path, start_time, end_time,
            os.path.join(output_dir, '{}_$index.jpg'.format(i + 1)),
            1
        )
        scene['snapshots'] = snapshots


dirname, basename = os.path.split(VIDEO_FILE)
basename = os.path.splitext(basename)[0]
images_dir = os.path.join(
    '..',
    'tmp',
    'images',
    os.path.relpath(dirname, VIDEO_DIR),
    basename
)
rmtree(images_dir, ignore_errors=True)

take_scenes_images(VIDEO_FILE, scenes, images_dir)

###############################
t0 = time()
with msg.loading('Plotting ...'):
    n_cols= 10
    n_rows = int(np.ceil(len(scenes) / n_cols))
    im_dsp_w = 80
    im_dsp_h = 60

    fig = plt.figure(figsize=(im_dsp_w, im_dsp_h))
    grid = ImageGrid(
        fig, len(scenes),  # similar to subplot(142)
        nrows_ncols=(n_rows, n_cols),
    )

    text_prop={'fontproperties': mfm_font_prop}
    for ax, scene in tqdm(zip(grid, scenes), total=len(scenes)):
        inner_title = f"场景[{scene['index']}]\nfr: {timedelta(seconds=scene['start_time'])}\nto: {timedelta(seconds=scene['end_time'])}"
        t = add_inner_title(ax, inner_title, loc=2)
        t.patch.set_alpha(0.5)
        im = mpimg.imread(scene['snapshots'][0])
        ax.imshow(im, interpolation="nearest")

    plt.pause(0.001)
    plt.show()
    plt.pause(0.001)

    msg.good(f'dur={timedelta(seconds=time()-t0)}')


## 场景分类与组合

我们已经制作了一个小的，可进行对场景进行视频分类的预训练模型，它可以将场景分为：

* `0`: 无字幕内容
* `1`: 有字幕内容
* `2`: 演播室

之所以只进行如此简单的3个分类，是因为我们只需要在这个笔记本中进行几个单一功能的概念验证，选取少量数据进行标注和训练，可以节约时间。

下面，我们进行分类，并根据分类结果重新组合场景。

首先，进行上一步骤中划分的场景进行分类，并输出缩略图。

**执行代码：**

In [ ]:
msg.info('场景分类:')


import torch.nn as nn
from torchvision import datasets, models, transforms


IMAGE_CLASSES = {
    0: '无字幕内容',
    1: '有字幕内容',
    2: '演播室',
}


INPUT_SIZE = 224


data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(INPUT_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ]),
    'val': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.CenterCrop(INPUT_SIZE),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ]),
}


def load_image_classify_model(path):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    if device.type == 'cpu':
        model_ft = torch.load(path, map_location='cpu')
        if isinstance(model_ft, nn.DataParallel):
            model_ft = model_ft.module
    else:
        if torch.cuda.device_count() > 1:
            model_ft = torch.load(path)
            model_ft = model_ft.to(device)
    return model_ft



def classify_scenes(scenes, model):
    tsfrm = data_transforms['val']
    samples_iter = (
        tsfrm(transforms.ToPILImage()(skimage.io.imread(path)))
        for path in (
            scene['snapshots'][0]
            for scene in scenes
        )
    )
    with torch.no_grad():
        model = model.eval()
        batched_samples = torch.tensor([np.array(im) for im in samples_iter]).to(device)
        predict_result = model(batched_samples)
    
    for scores, scene in zip(predict_result, scenes):
        class_score = sorted(chain.from_iterable(zip(enumerate(scores))), key=lambda x: x[1], reverse=True)
        top_class, top_score = class_score[0]
        scene['class'] = top_class
        scene['class_name'] = IMAGE_CLASSES[top_class]

t0 = time()
with msg.loading('classifying ...'):
    image_classify_model_archive = '../models/squeezenet1_0-py36.pkl'
    image_classify_model = load_image_classify_model(image_classify_model_archive)
    classify_scenes(scenes, image_classify_model)
    msg.good(f'dur={timedelta(seconds=time()-t0)}')

######################################################
t0 = time()
with msg.loading('plotting ...'):
    n_cols= 10
    n_rows = int(np.ceil(len(scenes) / n_cols))
    im_dsp_w = 80
    im_dsp_h = 60

    fig = plt.figure(figsize=(im_dsp_w, im_dsp_h))
    grid = ImageGrid(
        fig, len(scenes),  # similar to subplot(142)
        nrows_ncols=(n_rows, n_cols),
    )

    for i, (ax, scene) in tqdm(enumerate(zip(grid, scenes)), total=len(scenes)):
        inner_title = f"{i+1}: {IMAGE_CLASSES[scene['class']]}\nfr: {timedelta(seconds=scene['start_time'])}\nto: {timedelta(seconds=scene['end_time'])}"
        t = add_inner_title(ax, inner_title, loc=2)
        t.patch.set_alpha(0.5)
        im = mpimg.imread(scene['snapshots'][0])
        ax.imshow(im, interpolation="nearest")

    plt.pause(0.001)
    %time plt.show()
    plt.pause(0.001)

    msg.good(f'dur={timedelta(seconds=time()-t0)}')


然后，我们按照视频场景分类信息重新组合视频片段。

**执行代码：**

In [ ]:
###################################################
msg.info('重组合视频片段:')


# 合并类型相同的连续场景，去掉第一个演播室之前的

merged_scenes = []

for scene in scenes:
    if not merged_scenes:
        merged_scenes.append(deepcopy(scene))
    else:
        if merged_scenes[-1]['class'] == scene['class']:
            merged_scenes[-1]['end_time'] = scene['end_time']
        else:
            merged_scenes.append(deepcopy(scene))

while merged_scenes[0]['class'] != 2:
    del merged_scenes[0]

for i, scene in enumerate(merged_scenes):
    scene['index'] = i

###############################################
##############################################################
# 合并为故事片段

stories = []

for scene in merged_scenes:
    scene_class = scene['class']
    has_subtitle = scene_class == 1  # 是否有字幕
    in_studio = scene_class == 2  #  是否在演播室
    d = deepcopy(scene)
    d['in_studio'] = in_studio
    d['has_subtitle'] = has_subtitle

    if not stories: # 第一个，总是添加
        stories.append(d)

    elif in_studio:  # 只要是演播室，就作为新的片段
        stories.append(d)

    elif has_subtitle:  #  有字幕，作为新的片段
        stories.append(d)

    else: # 这次没有字幕
        if stories[-1]['in_studio']:  # 上一个是演播室，作为新的片段
            stories.append(d)
        else:  # 否则，上一个有字幕，合并片段
            stories[-1]['end_time'] = scene['end_time']

## 删除片尾：
# del stories[-1]
###########################################################################################################################
# Segmenting AV clips
dirname, basename = os.path.split(VIDEO_FILE)
basename, extname = os.path.splitext(basename)
clips_dir = os.path.join(
    '..',
    'tmp',
    'clips',
    os.path.relpath(dirname, VIDEO_DIR),
    basename
)
rmtree(clips_dir, ignore_errors=True)

for i, story in tqdm(enumerate(stories), total=len(stories)):
    story['video_file'] = os.path.join(clips_dir, f'{i}{extname}')
    os.makedirs(os.path.dirname(story['video_file']), exist_ok=True)

    story['audio_file'] = os.path.join(clips_dir, f'{i}.opus')
    os.makedirs(os.path.dirname(story['audio_file']), exist_ok=True)

    trunc_av(VIDEO_FILE, story['start_time'], story['end_time'], story['video_file'])
    trunc_a(VIDEO_FILE, story['start_time'], story['end_time'], story['audio_file'])

###############
_stories = [m for m in stories if m['has_subtitle']]

t0 = time()
with msg.loading('plotting ...'):
    n_cols= 10
    n_rows = int(np.ceil(len(_stories) / n_cols))
    im_dsp_w = 80
    im_dsp_h = 60

    fig = plt.figure(figsize=(im_dsp_w, im_dsp_h))
    grid = ImageGrid(
        fig, len(scenes),  # similar to subplot(142)
        nrows_ncols=(n_rows, n_cols),
    )

    for i, (ax, story) in tqdm(enumerate(zip(grid, _stories)), total=len(_stories)):
        inner_title = f"片段 {i+1}\nfr: {timedelta(seconds=story['start_time'])}\nto: {timedelta(seconds=story['end_time'])}"
        t = add_inner_title(ax, inner_title, loc=2)
        t.patch.set_alpha(0.5)
        im = mpimg.imread(story['snapshots'][0])
        ax.imshow(im, interpolation="nearest")

    plt.pause(0.001)
    plt.show()
    plt.pause(0.001)

    msg.good(f'dur={timedelta(seconds=time()-t0)}')
    msg.good(f'共有： {len(_stories)}(条) 新闻片段')


为了便于观察，我们输出这几个片段的视频：

**执行代码：**

In [ ]:
_stories = [m for m in stories if m['has_subtitle']]
# display
for i, story in enumerate(_stories):
    msg.info(
        '片段 [{}]({} ~ {})'.format(
            i+1,
            timedelta(seconds=story['start_time']),
            timedelta(seconds=story['end_time']))
    )
    display(Video(story['video_file']))
    print('-' * 100)
    print()

## 语音识别

使用 *场景分类与组合* 一节中，被预测到是演播室的片段，将新闻主播的语音识别为文本

**执行代码：**

In [ ]:
msg.info('语音识别:')


import weblfasr_python3_demo as lfasr


_stories = [m for m in stories if m['in_studio']]

for story in tqdm(_stories):
    asr_req = lfasr.RequestApi(os.getenv("XFYUN_APP_ID"), os.getenv("XFYUN_SECRET_KEY"), story['audio_file'])
    res = asr_req.all_api_request()
    asr_res_data = json.loads(res['data'])
    story['asr-result'] = asr_res_data

msg.good('语音识别完成')

识别完成后，我们输出视频片段与识别结果，进行评估：

**执行代码：**

In [ ]:
for story in _stories:
    display(Video(story['video_file']))
    text = ' '.join(m['onebest'] for m in story['asr-result'])
    print('语音内容：')
    print(text)
    print()


## 字幕识别（新闻标题）

使用 *场景分类* 一节中，被预测到具有字幕的内容，将视频中的字幕识别为文本

**执行代码：**

In [ ]:
msg.info('字幕识别:')


SUBTITLE_H_PENCENTS = (0.8, 0.98)

def crop_for_subtitle(arr):
    h, _, _ = arr.shape
    h_fr, h_to = np.ceil(np.repeat(h, 2) * SUBTITLE_H_PENCENTS).astype(int)
    return arr[h_fr:h_to, :, :]


def im_to_b64(arr, ext_name='.jpeg'):
    from tempfile import mkstemp
    from base64 import b64encode
    
    _, path = mkstemp(suffix=ext_name)
    try:
        plt.imsave(path, arr)
        with open(path, 'rb') as fp:
            data = fp.read()
    finally:
        os.remove(path)
    return b64encode(data).decode()

def ocr_subtitle(im_path):
    url = os.getenv('OCR_API_URL')
    pid = os.getenv('OCR_API_PID')
    key = os.getenv('OCR_API_KEY')
    service = os.getenv('OCR_API_SERVICE')
    salt = os.getenv('OCR_API_SALT')

    def _md5(s):
        m = hashlib.md5()
        m.update(s.encode("utf8"))
        return m.hexdigest()
    
    arr = plt.imread(im_path)
    arr = crop_for_subtitle(arr)

    image_base64_str = im_to_b64(arr)
    image_short = image_base64_str[0:1024]
    sign = _md5(pid + service + salt + image_short + key)

    data = {
        'pid': pid,
        'service': service,
        'sign': sign,
        'salt': salt,
        'image': image_base64_str,
    }
    headers = {
        'accept': "application/json"
    }
    res = requests.request("POST", url, data=data, headers=headers)
    res = res.json()

    if(res["success"]) == 1:
        result = res['result']
        return result
    else:
        raise RuntimeError("OCR ERROR")

        
_stories = [m for m in stories if m['has_subtitle']]
        
for story in tqdm(_stories):
    im_path = story['snapshots'][0]
    story['ocr_result'] = ocr_subtitle(im_path)

msg.good('字幕识别完成')

识别完毕之后，我们输出结果文本与视频、图像，进行观察比对：

**执行代码：**

In [ ]:
for i, story in enumerate(_stories):
    display(Video(story['video_file']))

    print(f"片段 {i+1} ({timedelta(seconds=story['start_time'])} ～ {timedelta(seconds=story['end_time'])})")
    print('字幕：')
    ocr_result = story['ocr_result']
    print(' | '.join(m['content'].strip() for m in ocr_result))
    
    im_path = story['snapshots'][0]
    fig, ax = plt.subplots(1,1,figsize=(6,6))
    im = plt.imread(im_path)
    h, _, _ = im.shape
    ax.imshow(im)
    for m in ocr_result:
        text_rect = [list(map(int, s.split(','))) for s in m['frame']]
        left, bottom = text_rect[0][0], text_rect[3][1]
        width = text_rect[1][0] - text_rect[0][0]
        height = text_rect[3][1] - text_rect[0][1]
        bottom += h * SUBTITLE_H_PENCENTS[0] - height
        rect = mpatches.Rectangle((left, bottom),width,height,linewidth=2.5,edgecolor='yellow',facecolor='none')
        ax.add_patch(rect)

    plt.show()
    plt.pause(1)

    print()

## 最终结果呈现

最终，我们得到了新闻现场视屏的片段及其标题，演播室片段及其语音内容，下面把它们列表呈现出来。

**执行代码：**

In [ ]:
for story in stories:
    if story['in_studio']:
        display(HTML('<br/><hr/>'))
        msg.info('演播室', f"{timedelta(seconds=story['start_time'])} ～ {timedelta(seconds=story['end_time'])}")
        display(Video(story['video_file']))
        display(HTML('<strong><mark>语音：</mark></strong>'))
        text = ' '.join(m['onebest'] for m in story['asr-result'])
        display(HTML(f'<mark>{text}</mark>'))
    elif story['has_subtitle']:
        display(HTML('<hr/>'))
        msg.info('新闻内容', f"{timedelta(seconds=story['start_time'])} ～ {timedelta(seconds=story['end_time'])}")
        display(HTML('<strong><mark>标题：</mark></strong>'))
        text = max(
            (m['content'].strip() for m in story['ocr_result']),
            key=lambda s: len(s)
        )
        display(HTML(f'<mark>{text}</mark>'))
        display(Video(story['video_file']))
